In [7]:
import pyspark
import random
sc = pyspark.SparkContext(appName="Pi")

In [8]:
customersPath =  "Customers.txt"
customersRDD = sc.textFile(customersPath)
customerWatchedPath =  "CustomerWatched.txt"
customerWatchedRDD = sc.textFile(customerWatchedPath)
episodesPath =  "Episodes.txt"
episodesRDD = sc.textFile(episodesPath).cache()
tvSeriesPath =  "TVSeries.txt"
tvSeriesRDD = sc.textFile(tvSeriesPath)

output1 = "outPart1"
output2 = "outPart2"


In [9]:
# Part1
# epi
# SID15,2,7,The One with the Blackout,1994/11/03
# tv series
# SID15,Friends,Comedy

In [10]:
ComedyTVIds = tvSeriesRDD\
        .filter(lambda l: str(l.split(',')[2]) == "Comedy")\
        .map(lambda l: ((l.split(',')[0]), 1))

def episodesMap(l):
    id = l.split(',')[0]
    season = l.split(',')[1]
    return (id, season)
             
episodes = episodesRDD.map(episodesMap)
episodesComedy = episodes.join(ComedyTVIds)

In [11]:
episodesComedy.collect()

[('SID1', ('1', 1)),
 ('SID1', ('1', 1)),
 ('SID1', ('1', 1)),
 ('SID1', ('1', 1)),
 ('SID1', ('1', 1)),
 ('SID1', ('1', 1)),
 ('SID1', ('1', 1)),
 ('SID1', ('1', 1)),
 ('SID1', ('1', 1)),
 ('SID1', ('1', 1)),
 ('SID1', ('1', 1)),
 ('SID1', ('2', 1)),
 ('SID1', ('2', 1)),
 ('SID1', ('2', 1)),
 ('SID1', ('2', 1)),
 ('SID3', ('1', 1))]

In [12]:
res1 = episodesComedy\
        .map(lambda i: ((i[0], i[1][0]), i[1][1]))\
        .reduceByKey(lambda v1, v2: v1+v2)\
        .map(lambda i: (i[0][0], (i[1], 1)))\
        .reduceByKey(lambda t1, t2: (t1[0]+t2[0], t1[1]+t2[1]))\
        .map(lambda i: (i[0], i[1][0]/i[1][1]))

In [13]:
res1.collect()

[('SID1', 7.5), ('SID3', 1.0)]

In [15]:
# Part2 
# Epis
# SID15,2,7,The One with the Blackout,1994/11/03
# CustomerWatched
# CID10,2022/11/07-21:40,SID15,1,7

pairTVSSeason = episodesRDD\
            .map(lambda l: (l.split(',')[0], l.split(',')[1]))\
            .distinct()\
            .map(lambda elem: (elem[0], 1))\
            .reduceByKey(lambda v1, v2: v1 + v2)\
            .map(lambda elem: (elem, None))

pairTVSSeasonCustomer = customerWatchedRDD\
                    .map(lambda l: (l.split(',')[2], l.split(',')[3], l.split(',')[0]))\
                    .distinct()\
                    .map(lambda elem: ((elem[0], elem[2]), 1))\
                    .reduceByKey(lambda v1, v2: v1 + v2)\
                    .map(lambda elem: ((elem[0][0], elem[1]), elem[0][1]))

In [17]:
pairTVSSeason.collect()

[(('SID1', 2), None), (('SID2', 2), None), (('SID3', 1), None)]

In [18]:
pairTVSSeasonCustomer.collect()

[(('SID1', 1), 'CID1'),
 (('SID2', 2), 'CID2'),
 (('SID1', 1), 'CID3'),
 (('SID3', 1), 'CID4'),
 (('SID1', 1), 'CID2'),
 (('SID3', 1), 'CID1')]

In [25]:
res2 = pairTVSSeason\
    .join(pairTVSSeasonCustomer)\
    .map(lambda elem: (elem[0][0] ,elem[1][1]))

In [26]:
res2.collect()

[('SID3', 'CID4'), ('SID3', 'CID1'), ('SID2', 'CID2')]

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 34132)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
                           ^^^^^^
  File "/usr/local/spark/python/pyspark/accumulators.py", line 271, in accum_updates
    num_updates =